Importações e Requisitos

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
import os
import re

Setup Inicial e Passar Imagens para uma lista

In [ ]:
# Setup Inicial
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("--incognito") # Abrir em modo anônimo para evitar cookies/cache problemáticos

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install(), log_level='OFF'), options=options)
driver.get("https://www.instagram.com/accounts/login/")

time.sleep(5)

# Login
username = driver.find_element(By.NAME, "username")
password = driver.find_element(By.NAME, "password")
username.send_keys("your username")
password.send_keys("your password")
password.send_keys(Keys.RETURN)

time.sleep(7) # Aumentar o tempo de espera após o login

# Acessar aba Salvos
driver.get("https://www.instagram.com/enriquesusin/saved/rpg/17948031698580212/")
time.sleep(7) # Aumentar o tempo de espera para a página de salvos carregar

post_urls = set()
scroll_attempts = 0
max_scroll_attempts = 15 # Aumentar o limite de tentativas

print("Iniciando coleta de URLs de postagens salvas...")
while scroll_attempts < max_scroll_attempts:
    initial_post_count = len(post_urls)

    # Coletar URLs das postagens
    # Os links para posts são geralmente encontrados em divs com role="link" ou tags <a>
    # O Instagram usa o atributo 'href' para o link do post
    post_links = driver.find_elements(By.XPATH, "//a[contains(@href, '/p/')]")

    for link in post_links:
        href = link.get_attribute("href")
        if href and "/p/" in href:
            post_urls.add(href)

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4) # Tempo para carregar mais postagens

    # Re-coletar URLs após a rolagem para verificar se novos posts apareceram
    post_links_after_scroll = driver.find_elements(By.XPATH, "//a[contains(@href, '/p/')]")
    new_urls_found = False
    for link in post_links_after_scroll:
        href = link.get_attribute("href")
        if href and "/p/" in href and href not in post_urls:
            post_urls.add(href)
            new_urls_found = True

    if not new_urls_found: # Se nenhuma nova URL foi encontrada após a rolagem
        scroll_attempts += 1
    else:
        scroll_attempts = 0 # Reseta se novas URLs foram encontradas

    print(f"Número atual de URLs de postagens coletadas: {len(post_urls)}")

print(f"Total de URLs de postagens únicas encontradas: {len(post_urls)}")

driver.quit()

Iniciando coleta de URLs de postagens salvas...
Número atual de URLs de postagens coletadas: 54


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=137.0.7151.120)
Stacktrace:
	GetHandleVerifier [0x0xe43b03+62899]
	GetHandleVerifier [0x0xe43b44+62964]
	(No symbol) [0x0xc710f3]
	(No symbol) [0x0xc608c0]
	(No symbol) [0x0xc7e87f]
	(No symbol) [0x0xce514c]
	(No symbol) [0x0xcff6a9]
	(No symbol) [0x0xcde306]
	(No symbol) [0x0xcad670]
	(No symbol) [0x0xcae4e4]
	GetHandleVerifier [0x0x10a4793+2556483]
	GetHandleVerifier [0x0x109fd02+2537394]
	GetHandleVerifier [0x0xe6a2fa+220586]
	GetHandleVerifier [0x0xe5aae8+157080]
	GetHandleVerifier [0x0xe6141d+184013]
	GetHandleVerifier [0x0xe4ba68+95512]
	GetHandleVerifier [0x0xe4bc10+95936]
	GetHandleVerifier [0x0xe36b5a+9738]
	BaseThreadInitThunk [0x0x7681fcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x77de82ae+286]
	RtlGetAppContainerNamedObjectPath [0x0x77de827e+238]


In [ ]:
# Lista para armazenar todas as URLs de imagem/vídeo em alta qualidade
all_media_urls = []

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install(), log_level='OFF'), options=options)
driver.get("https://www.instagram.com/accounts/login/")

time.sleep(5)

# Login
username = driver.find_element(By.NAME, "username")
password = driver.find_element(By.NAME, "password")
username.send_keys("your username")
password.send_keys("your password")
password.send_keys(Keys.RETURN)

time.sleep(7)

print("\nIniciando extração de mídias de cada postagem...")

# Iterar sobre cada URL de postagem e extrair as mídias
for i, post_url in enumerate(list(post_urls)):
    print(f"Acessando postagem {i+1}/{len(post_urls)}: {post_url}")
    driver.get(post_url)
    time.sleep(8)

    current_post_media = set()

    # Função auxiliar para coletar mídias de uma página
    def collect_media(driver_instance, media_set):
        # Manter os seletores originais que funcionavam
        media_elements = driver_instance.find_elements(By.CSS_SELECTOR, 
            "main div._aagv > img, main div._aagv > video, main img._aagv, main video._aagv")

        # Seletor adicional mais amplo para garantir que pegamos todas as mídias do post
        additional_media = driver_instance.find_elements(By.CSS_SELECTOR,
            "main article img, main article video")
        
        # Combinar os elementos encontrados
        all_elements = media_elements + additional_media
        
        for element in all_elements:
            src = element.get_attribute("src")
            if src:
                # Filtros básicos originais
                if (
                    "profile_pic" in src
                    or "/s150x150/" in src
                    or "/s320x320/" in src
                    or "/_nc_d/" in src
                    or "data:image" in src
                    or "svg" in src
                ):
                    continue

                # NOVO: Verificar se o elemento está na área de sugestões
                # Pegar a posição Y do elemento na página
                try:
                    element_y = element.location['y']
                    # Se o elemento estiver muito abaixo (área de sugestões), pular
                    # Valor ajustável - as sugestões geralmente aparecem depois da posição 1500px
                    if element_y > 1500:
                        continue
                except:
                    pass

                # NOVO: Verificar se o elemento tem tamanho muito pequeno (avatares/ícones)
                try:
                    size = element.size
                    if size['width'] < 50 or size['height'] < 50:
                        continue
                except:
                    pass

                # NOVO: Verificar se não está em um container de sugestões
                try:
                    # Executar JavaScript para verificar se o elemento está em uma área de sugestões
                    is_in_suggestions = driver_instance.execute_script("""
                        var element = arguments[0];
                        var current = element;
                        
                        // Subir na hierarquia procurando por indicadores de sugestões
                        for (let i = 0; i < 10; i++) {
                            if (!current.parentElement) break;
                            current = current.parentElement;
                            
                            // Verificar atributos e classes que indicam área de sugestões
                            var classList = current.className || '';
                            var textContent = current.textContent || '';
                            
                            // Indicadores comuns de área de sugestões
                            if (classList.includes('_ac69') || 
                                classList.includes('_ac6a') ||
                                textContent.includes('Sugestões para você') ||
                                textContent.includes('Suggestions for you') ||
                                textContent.includes('More posts from') ||
                                textContent.includes('Mais publicações de')) {
                                return true;
                            }
                        }
                        return false;
                    """, element)
                    
                    if is_in_suggestions:
                        continue
                except:
                    pass

                # Se passou por todos os filtros, adicionar à lista
                if element.tag_name == 'video':
                    media_set.add(src)
                elif element.tag_name == 'img':
                    media_set.add(src)
                    
        return media_set

    # Coleta inicial de mídias
    current_post_media = collect_media(driver, current_post_media)

    # Lidar com carrosséis - MANTER A LÓGICA ORIGINAL QUE FUNCIONAVA
    next_button_selector = "button[aria-label='Avançar'], button[aria-label='Próximo'], button svg[aria-label='Avançar'], button svg[aria-label='Próximo']"
    max_carousel_attempts = 15

    carousel_attempt_count = 0
    while carousel_attempt_count < max_carousel_attempts:
        try:
            next_buttons = driver.find_elements(By.CSS_SELECTOR, next_button_selector)
            if not next_buttons:
                break

            # Clicar no primeiro botão "Avançar" visível e interativo
            driver.execute_script("arguments[0].click();", next_buttons[0])
            time.sleep(4)

            # Coletar novas mídias após o clique
            new_media_count = len(current_post_media)
            current_post_media = collect_media(driver, current_post_media)

            if len(current_post_media) == new_media_count:
                carousel_attempt_count += 1
            else:
                carousel_attempt_count = 0

        except Exception as e:
            print(f"  Erro ao navegar no carrossel ou coletar mídia: {e}")
            break

    all_media_urls.extend(list(current_post_media))
    print(f"  Encontradas {len(current_post_media)} mídias nesta postagem.")

print(f"\nTotal de mídias coletadas: {len(all_media_urls)}")
driver.quit()


Iniciando extração de mídias de cada postagem...
Acessando postagem 1/54: https://www.instagram.com/p/DJ9Cpy3tlcu/
  Encontradas 5 mídias nesta postagem.
Acessando postagem 2/54: https://www.instagram.com/p/DKp8OMKR-hN/
  Encontradas 1 mídias nesta postagem.
Acessando postagem 3/54: https://www.instagram.com/p/DImHEnyIIEe/
  Encontradas 2 mídias nesta postagem.
Acessando postagem 4/54: https://www.instagram.com/p/DKchhp7qRcY/


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=137.0.7151.120)
Stacktrace:
	GetHandleVerifier [0x0xe43b03+62899]
	GetHandleVerifier [0x0xe43b44+62964]
	(No symbol) [0x0xc710f3]
	(No symbol) [0x0xc608c0]
	(No symbol) [0x0xc7e87f]
	(No symbol) [0x0xce514c]
	(No symbol) [0x0xcff6a9]
	(No symbol) [0x0xcde306]
	(No symbol) [0x0xcad670]
	(No symbol) [0x0xcae4e4]
	GetHandleVerifier [0x0x10a4793+2556483]
	GetHandleVerifier [0x0x109fd02+2537394]
	GetHandleVerifier [0x0xe6a2fa+220586]
	GetHandleVerifier [0x0xe5aae8+157080]
	GetHandleVerifier [0x0xe6141d+184013]
	GetHandleVerifier [0x0xe4ba68+95512]
	GetHandleVerifier [0x0xe4bc10+95936]
	GetHandleVerifier [0x0xe36b5a+9738]
	BaseThreadInitThunk [0x0x7681fcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x77de82ae+286]
	RtlGetAppContainerNamedObjectPath [0x0x77de827e+238]


Passar a Imagens para txt

In [53]:
# Remover URLs duplicadas e salvar em TXT
all_media_urls = list(set(all_media_urls)) # Garante que todas as URLs são únicas

output_file_name = "urls_instagram_media_hq.txt"
with open(output_file_name, "w") as file:
    for url in all_media_urls:
        file.write(url + "\n")
print(f"Total de {len(all_media_urls)} URLs de mídia únicas salvas em '{output_file_name}'.")


Total de 8 URLs de mídia únicas salvas em 'urls_instagram_media_hq.txt'.


Baixar as Imagens do Arquivo Txt

In [54]:
# Baixar as Mídias do Arquivo Txt
folder_name = "midias_salvas_instagram_hq"
os.makedirs(folder_name, exist_ok=True)

print(f"\nIniciando download das mídias para a pasta '{folder_name}'...")
with open(output_file_name, "r") as file:
    urls_to_download = file.readlines()

for i, url in enumerate(urls_to_download):
    url = url.strip()
    if not url:
        continue

    try:
        response = requests.get(url, stream=True, timeout=10) # Adicionado timeout
        response.raise_for_status()

        # Determinar a extensão do arquivo
        if ".mp4" in url:
            ext = ".mp4"
        elif ".jpg" in url or ".jpeg" in url:
            ext = ".jpg"
        elif ".png" in url:
            ext = ".png"
        elif ".webp" in url:
            ext = ".webp"
        else:
            ext = ".unknown" # Caso não consiga determinar

        # Tentar extrair um nome de arquivo da URL ou usar um nome genérico
        filename_match = re.search(r'/([a-zA-Z0-9_-]+\.(?:jpg|jpeg|png|gif|webp|mp4))(?:\?|$)', url)
        if filename_match:
            original_filename = filename_match.group(1)
            filename = os.path.join(folder_name, original_filename)
        else:
            filename = os.path.join(folder_name, f"media_{i + 1}{ext}") # Usa a extensão determinada

        # Evitar sobrescrever arquivos se já existirem nomes iguais
        base_filename, file_extension = os.path.splitext(filename)
        counter = 1
        while os.path.exists(filename):
            filename = f"{base_filename}_{counter}{file_extension}"
            counter += 1

        with open(filename, "wb") as media_file:
            for chunk in response.iter_content(1024):
                media_file.write(chunk)

        print(f"Mídia {i + 1} baixada com sucesso: {filename}")

    except requests.exceptions.RequestException as e:
        print(f"Erro ao baixar {url}: {e}")
    except Exception as e:
        print(f"Erro inesperado ao processar {url}: {e}")

print("Download concluído!")


Iniciando download das mídias para a pasta 'midias_salvas_instagram_hq'...
Mídia 1 baixada com sucesso: midias_salvas_instagram_hq\499574083_18458772367076287_5752826221385983875_n.jpg
Mídia 2 baixada com sucesso: midias_salvas_instagram_hq\500406608_18458772385076287_7971237191225504271_n.jpg
Mídia 3 baixada com sucesso: midias_salvas_instagram_hq\500644801_18458772421076287_7226456963454330702_n.jpg
Mídia 4 baixada com sucesso: midias_salvas_instagram_hq\498328425_18458772412076287_3634416194678942237_n.jpg
Mídia 5 baixada com sucesso: midias_salvas_instagram_hq\490423582_18381551989184104_7345559584130859450_n.webp
Mídia 6 baixada com sucesso: midias_salvas_instagram_hq\499405591_18458772403076287_6386946644443369952_n.jpg
Mídia 7 baixada com sucesso: midias_salvas_instagram_hq\491170742_18381551998184104_8708980706082574707_n.webp
Mídia 8 baixada com sucesso: midias_salvas_instagram_hq\504203832_18049370741387192_5272494723686747428_n.webp
Download concluído!
